In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

# データの準備
# ひらがな「み」「か」「た」の対応表
char_to_idx = {'み': 0, 'か': 1, 'た': 2}
idx_to_char = {v: k for k, v in char_to_idx.items()}

# シーケンスデータの作成
sequences = [
    ['み', 'か', 'た'],
    ['か', 'た', 'み'],
    ['た', 'み', 'か']
]

# 入力とラベルを数値に変換
X = []
y = []

for seq in sequences:
    X.append([char_to_idx[char] for char in seq[:-1]])  # 最後の文字を除く
    y.append(char_to_idx[seq[-1]])  # 最後の文字

X = np.array(X)
y = np.array(y)

# One-hotエンコーディング
X = to_categorical(X, num_classes=len(char_to_idx))
y = to_categorical(y, num_classes=len(char_to_idx))

# モデルの構築
model = Sequential([
    SimpleRNN(10, input_shape=(X.shape[1], X.shape[2]), activation='tanh'),
    Dense(len(char_to_idx), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 学習
model.fit(X, y, epochs=500, verbose=0)

# 予測
def predict_next_char(input_seq):
    input_encoded = [char_to_idx[char] for char in input_seq]
    input_encoded = to_categorical(input_encoded, num_classes=len(char_to_idx))
    input_encoded = input_encoded[np.newaxis, ...]  # バッチ次元を追加
    prediction = model.predict(input_encoded, verbose=0)
    predicted_idx = np.argmax(prediction)
    return idx_to_char[predicted_idx]

# テスト
test_input = ['み', 'か']  # 例えば「み」「か」と入力
predicted_char = predict_next_char(test_input)
print(f"入力: {test_input} -> 次の文字の予測: {predicted_char}")


入力: ['み', 'か'] -> 次の文字の予測: た


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
import tkinter as tk

# データの準備
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sequences = [line.strip() for line in f if line.strip()]
    return sequences

# 学習データの読み込み
file_path = 'data.txt'
sequences = load_data(file_path)

# ユニークな文字を取得
unique_chars = sorted(set(''.join(sequences)))
char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# シーケンスデータの作成
X = []
y = []

for seq in sequences:
    for i in range(len(seq) - 1):
        X.append([char_to_idx[char] for char in seq[i:i+2]])  # 2文字ずつ
        y.append(char_to_idx[seq[i+2]] if i + 2 < len(seq) else char_to_idx[seq[0]])  # 次の文字 or 循環

    # 1文字で次を予測するケース
    for i in range(len(seq)):
        X.append([char_to_idx[seq[i]], char_to_idx[seq[i]]])  # 同じ文字を2回
        y.append(char_to_idx[seq[(i + 1) % len(seq)]])

X = np.array(X)
y = np.array(y)

# One-hotエンコーディング
X = to_categorical(X, num_classes=len(char_to_idx))
y = to_categorical(y, num_classes=len(char_to_idx))

# モデルの構築
model = Sequential([
    SimpleRNN(10, input_shape=(X.shape[1], X.shape[2]), activation='tanh'),
    Dense(len(char_to_idx), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=500, verbose=0)

# 予測関数
def predict_next_char(input_seq):
    # 学習データにない文字が入力されている場合
    if any(char not in char_to_idx for char in input_seq):
        return "未対応の文字が含まれています"
    
    # 入力が2文字未満の場合、補完
    if len(input_seq) < 2:
        input_seq = input_seq * 2
    
    # 最後の2文字を使用して予測
    input_encoded = [char_to_idx[char] for char in input_seq[-2:]]
    input_encoded = to_categorical(input_encoded, num_classes=len(char_to_idx))
    input_encoded = input_encoded[np.newaxis, ...]
    prediction = model.predict(input_encoded, verbose=0)
    predicted_idx = np.argmax(prediction)
    return idx_to_char[predicted_idx]

# TkinterによるGUI
def on_input_change(event):
    input_text = input_field.get()
    if len(input_text) == 0:
        result_label.config(text="文字を入力してください")
        return

    predicted_char = predict_next_char(input_text)
    result_label.config(text=f"次の文字の予測: {predicted_char}")

# ウィンドウの作成
window = tk.Tk()
window.title("文字予測アプリ")

# ラベル
instruction_label = tk.Label(window, text="任意の長さの文字を入力してください")
instruction_label.pack()

# 入力フィールド
input_field = tk.Entry(window)
input_field.pack()

# 入力フィールドの変更時に予測を実行
input_field.bind("<KeyRelease>", on_input_change)

# 結果表示ラベル
result_label = tk.Label(window, text="")
result_label.pack()

# アプリ実行
window.mainloop()


1. モデル学習・保存部分 (train_model.py)

In [60]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

# データの準備
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sequences = [line.strip() for line in f if line.strip()]
    return sequences

# シーケンスデータの作成
def create_training_data(sequences, char_to_idx):
    X, y = [], []
    for seq in sequences:
        for i in range(len(seq) - 1):
            X.append([char_to_idx[char] for char in seq[i:i+2]])  # 2文字ずつ
            y.append(char_to_idx[seq[i+2]] if i + 2 < len(seq) else char_to_idx[seq[0]])  # 次の文字 or 循環

        # 1文字で次を予測するケース
        for i in range(len(seq)):
            X.append([char_to_idx[seq[i]], char_to_idx[seq[i]]])  # 同じ文字を2回
            y.append(char_to_idx[seq[(i + 1) % len(seq)]])
    return np.array(X), np.array(y)

# モデルの構築
def build_model(num_classes):
    model = Sequential([
        SimpleRNN(10, input_shape=(2, num_classes), activation='tanh'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

if __name__ == "__main__":
    # 学習データの読み込み
    file_path = 'data.txt'
    sequences = load_data(file_path)

    # ユニークな文字を取得
    unique_chars = sorted(set(''.join(sequences)))
    char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}

    # データ作成
    X, y = create_training_data(sequences, char_to_idx)
    X = to_categorical(X, num_classes=len(char_to_idx))
    y = to_categorical(y, num_classes=len(char_to_idx))

    # モデルの作成と学習
    model = build_model(len(char_to_idx))
    model.fit(X, y, epochs=500, verbose=1)

    # モデル保存
    model.save('char_prediction_model.h5')
    with open('char_mapping.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(unique_chars))


Epoch 1/500
2/2 [==============================] - 0s 2ms/step - loss: 2.9540 - accuracy: 0.0976
Epoch 2/500
2/2 [==============================] - 0s 3ms/step - loss: 2.9443 - accuracy: 0.0976
Epoch 3/500
2/2 [==============================] - 0s 3ms/step - loss: 2.9367 - accuracy: 0.0976
Epoch 4/500
2/2 [==============================] - 0s 3ms/step - loss: 2.9285 - accuracy: 0.0976
Epoch 5/500
2/2 [==============================] - 0s 1ms/step - loss: 2.9206 - accuracy: 0.0976
Epoch 6/500
2/2 [==============================] - 0s 2ms/step - loss: 2.9130 - accuracy: 0.1220
Epoch 7/500
2/2 [==============================] - 0s 2ms/step - loss: 2.9053 - accuracy: 0.1220
Epoch 8/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8979 - accuracy: 0.1220
Epoch 9/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8903 - accuracy: 0.1220
Epoch 10/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8828 - accuracy: 0.1220
Epoch 11/500
2/2 [===========

2. モデル実行・追加学習部分 (run_model.py)

In [65]:
import numpy as np
from keras.models import load_model, Sequential
from keras.layers import SimpleRNN, Dense
from keras.utils import to_categorical
import tkinter as tk

# マッピングの読み込み
def load_mappings(mapping_file):
    with open(mapping_file, 'r', encoding='utf-8') as f:
        unique_chars = [line.strip() for line in f]
    char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}
    return unique_chars, char_to_idx, idx_to_char

# マッピングの更新
def update_mappings(new_chars, mapping_file):
    with open(mapping_file, 'a', encoding='utf-8') as f:
        for char in new_chars:
            f.write(f"{char}\n")

# モデルの再構築
def rebuild_model(model, num_classes, added_classes):
    rnn_weights = model.layers[0].get_weights()
    dense_weights = model.layers[1].get_weights()

    # RNN層の重みを拡張
    rnn_weights[0] = np.pad(rnn_weights[0], ((0, added_classes), (0, 0)), mode='constant')
    rnn_weights[1] = np.pad(rnn_weights[1], ((0, 0), (0, added_classes)), mode='constant')
    rnn_weights[2] = np.pad(rnn_weights[2], (0, added_classes), mode='constant')

    # Dense層の重みを拡張
    dense_weights[0] = np.pad(dense_weights[0], ((0, 0), (0, added_classes)), mode='constant')
    dense_weights[1] = np.pad(dense_weights[1], (0, added_classes), mode='constant')

    # 新しいモデルを構築
    new_model = Sequential([
        SimpleRNN(10, input_shape=(2, num_classes), activation='tanh'),
        Dense(num_classes, activation='softmax')
    ])
    new_model.layers[0].set_weights(rnn_weights)
    new_model.layers[1].set_weights(dense_weights)
    new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return new_model

# データセットの作成
def create_additional_data(sequences, char_to_idx):
    X, y = [], []
    for sequence in sequences:
        for i in range(len(sequence) - 2):
            X.append([char_to_idx[sequence[i]], char_to_idx[sequence[i + 1]]])
            y.append(char_to_idx[sequence[i + 2]])
    return np.array(X), np.array(y)

# 文字予測
def predict_next_char(model, input_seq, char_to_idx, idx_to_char):
    if any(char not in char_to_idx for char in input_seq):
        return "未対応の文字が含まれています"
    if len(input_seq) < 2:
        input_seq = input_seq * 2
    input_encoded = [char_to_idx[char] for char in input_seq[-2:]]
    input_encoded = to_categorical(input_encoded, num_classes=len(char_to_idx))
    input_encoded = input_encoded[np.newaxis, ...]
    prediction = model.predict(input_encoded, verbose=0)
    predicted_idx = np.argmax(prediction)
    return idx_to_char[predicted_idx]

# モデルとマッピングファイルの読み込み
model = load_model('char_prediction_model.h5')
unique_chars, char_to_idx, idx_to_char = load_mappings('char_mapping.txt')

# Tkinter GUI
def on_key_release(event=None):
    input_seq = input_field.get()
    if input_seq:
        prediction = predict_next_char(model, input_seq, char_to_idx, idx_to_char)
        result_label.config(text=f"次の文字予測: {prediction}")
    else:
        result_label.config(text="次の文字予測: ")

def on_add_training_data():
    global model, char_to_idx, idx_to_char
    new_sequence = input_field.get()
    if len(new_sequence) < 2:
        result_label.config(text="2文字以上のシーケンスを入力してください")
        return

    # 入力データを `data.txt` に追記
    with open('data.txt', 'a', encoding='utf-8') as f:
        f.write(new_sequence + '\n')

    # 新しい文字を検出
    new_chars = [char for char in new_sequence if char not in char_to_idx]
    if new_chars:
        result_label.config(text=f"新しい文字を検出: {', '.join(new_chars)}")
        update_mappings(new_chars, 'char_mapping.txt')
        unique_chars, char_to_idx, idx_to_char = load_mappings('char_mapping.txt')
        model = rebuild_model(model, len(char_to_idx), len(new_chars))

    # 追加学習
    X_new, y_new = create_additional_data([new_sequence], char_to_idx)
    X_new = to_categorical(X_new, num_classes=len(char_to_idx))
    y_new = to_categorical(y_new, num_classes=len(char_to_idx))
    model.fit(X_new, y_new, epochs=50, verbose=1)

    # モデルとマッピングの保存
    model.save('char_prediction_model_updated.h5')
    with open('char_mapping.txt', 'w', encoding='utf-8') as f:
        for char in char_to_idx.keys():
            f.write(f"{char}\n")

    result_label.config(text="データ追記と追加学習が完了しました！")

def show_current_files():
    result_label.config(text="現在のモデル: char_prediction_model.h5\nマッピングファイル: char_mapping.txt")

root = tk.Tk()
root.title("文字予測モデル")

input_field = tk.Entry(root, width=30)
input_field.grid(row=0, column=0, padx=10, pady=10)
input_field.bind('<KeyRelease>', on_key_release)  # 入力フィールドに文字が入力されるたびに呼び出し

train_button = tk.Button(root, text="追加学習", command=on_add_training_data)
train_button.grid(row=1, column=0, padx=10, pady=10)

show_button = tk.Button(root, text="現在のファイル", command=show_current_files)
show_button.grid(row=1, column=1, padx=10, pady=10)

result_label = tk.Label(root, text="")
result_label.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

root.mainloop()


追加学習

In [56]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

# データの読み込み
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sequences = [line.strip() for line in f if line.strip()]
    return sequences

# シーケンスデータの作成
def create_training_data(sequences, char_to_idx):
    X, y = [], []
    for seq in sequences:
        for i in range(len(seq) - 1):
            X.append([char_to_idx[char] for char in seq[i:i+2]])  # 2文字ずつ
            y.append(char_to_idx[seq[i+2]] if i + 2 < len(seq) else char_to_idx[seq[0]])  # 次の文字 or 循環
    return np.array(X), np.array(y)

# マッピングの読み込み
def load_mappings(mapping_file):
    with open(mapping_file, 'r', encoding='utf-8') as f:
        unique_chars = [line.strip() for line in f]
    char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}
    return unique_chars, char_to_idx, idx_to_char

# マッピングの更新
def update_mappings(new_chars, mapping_file):
    with open(mapping_file, 'a', encoding='utf-8') as f:
        for char in new_chars:
            f.write(f"{char}\n")

# モデルの再構築
def rebuild_model(model, num_classes, added_classes):
    rnn_weights = model.layers[0].get_weights()
    dense_weights = model.layers[1].get_weights()

    # RNN層の重みを拡張
    rnn_weights[0] = np.pad(rnn_weights[0], ((0, added_classes), (0, 0)), mode='constant')
    rnn_weights[1] = np.pad(rnn_weights[1], ((0, 0), (0, added_classes)), mode='constant')
    rnn_weights[2] = np.pad(rnn_weights[2], (0, added_classes), mode='constant')

    # Dense層の重みを拡張
    dense_weights[0] = np.pad(dense_weights[0], ((0, 0), (0, added_classes)), mode='constant')
    dense_weights[1] = np.pad(dense_weights[1], (0, added_classes), mode='constant')

    # 新しいモデルを構築
    new_model = Sequential([
        SimpleRNN(10, input_shape=(2, num_classes), activation='tanh'),
        Dense(num_classes, activation='softmax')
    ])
    new_model.build(input_shape=(None, 2, num_classes))
    new_model.layers[0].set_weights(rnn_weights)
    new_model.layers[1].set_weights(dense_weights)
    new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return new_model

if __name__ == "__main__":
    # 既存モデルとマッピングの読み込み
    model_path = 'char_prediction_model.h5'
    mapping_path = 'char_mapping.txt'
    additional_data_path = 'data1.txt'

    model = load_model(model_path)
    unique_chars, char_to_idx, idx_to_char = load_mappings(mapping_path)

    # 新しいデータの読み込み
    new_sequences = load_data(additional_data_path)

    # 新しい文字を検出してマッピングを更新
    new_chars = sorted(set(''.join(new_sequences)) - set(unique_chars))
    if new_chars:
        print(f"新しい文字を検出: {', '.join(new_chars)}")
        update_mappings(new_chars, mapping_path)
        unique_chars, char_to_idx, idx_to_char = load_mappings(mapping_path)
        model = rebuild_model(model, len(char_to_idx), len(new_chars))

    # 新しいデータで学習
    X_new, y_new = create_training_data(new_sequences, char_to_idx)
    
    # X_new の各シーケンスをワンホットエンコードに変換
    X_new = np.array([to_categorical(seq, num_classes=len(char_to_idx)) for seq in X_new])  # (サンプル数, 2, num_classes)
    y_new = to_categorical(y_new, num_classes=len(char_to_idx))  # ワンホットエンコード

    # モデルの入力形状を確認
    print(f"X_new shape: {X_new.shape}, Model expects: {model.input_shape}")

    # 追加学習
    print(f"追加学習を開始します: データ数={len(X_new)}")
    model.fit(X_new, y_new, epochs=50, verbose=1)

    # 更新したモデルとマッピングを保存
    updated_model_path = 'char_prediction_model_updated.h5'
    model.save(updated_model_path)
    print(f"追加学習が完了しました。モデルを {updated_model_path} に保存しました。")



X_new shape: (11, 2, 23), Model expects: (None, 2, 17)
追加学習を開始します: データ数=11
Epoch 1/50


ValueError: in user code:

    File "C:\Users\t-sat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\t-sat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\t-sat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\t-sat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 993, in train_s